## Título
Desenvolver uma análise quantitativa e gráfica relacionando o aumento das ações da
Petrobrás (PETR4) com as variações da taxa SELIC para os últimos 10 anos.

## Objetivo
Fazer um programa em python para:
- Extrair as informações dos sites: https://www.infomoney.com.br/petrobras-petr4/cotacoes e 
https://www.bcb.gov.br/pec/copom/port/taxaselic.asp e tratá-las;
- Armazenar as informações tratadas no SQLite;
  - Utilizar, de preferência, ORM (object relational mapping)
- Ler as informações do banco e apresentar em formato gráfico.

## Restrições
- Utilizar a biblioteca pandas do Python

---

## ETL
- Vou utilizar a granularidade mensal para a análise.
- Será feito 2 scrapping. Para garantir um serviço menos monolítico farei cada scrapping separado. Desta forma é possível garantir uma melhor manutenção de código.

In [7]:
! python3 etl_PETR4.py

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4963 entries, 0 to 0
Data columns (total 3 columns):
Ano           4963 non-null int16
Mes           4963 non-null int8
Fechamento    4962 non-null float32
dtypes: float32(1), int16(1), int8(1)
memory usage: 72.7 KB
None 

(df_petr4.csv) salvo!


In [8]:
! python3 etl_SELIC.py

python3: can't open file 'etl_SELIC.py': [Errno 2] No such file or directory


### Load